In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk
import torchvision.transforms as transforms
import json
import os
import re
import datasets
import numpy as np
from transformers import VisionEncoderDecoderModel, AutoTokenizer, ViTModel, ViTImageProcessor, ViTFeatureExtractor, AutoModelForCausalLM, AutoTokenizer
import wandb
from PIL import Image
import torch

c:\Users\achia\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
# raw_train_dataset = load_from_disk("processed_train")
# raw_val_dataset = load_from_disk("processed_val")

In [4]:
# train_dataset = raw_val_dataset.select(range(1000))
# val_dataset = raw_val_dataset.select(range(100,1125))

# ds = DatasetDict({'train':train_dataset, 'val':val_dataset})
# ds

In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side='left')
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# ViT =  ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
model = AutoModelForCausalLM.from_pretrained("gpt2", max_length=256,)

c:\Users\achia\anaconda3\envs\llm\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [6]:
# model.config.decoder_start_token_id = tokenizer.cls_token_id
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# model.config.vocab_size = model.config.decoder.vocab_size

model.config.eos_token_id = tokenizer.eos_token_id
# model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.max_length = 256
model.generation_config.max_length = 256
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

model.generation_config.pad_token_id = model.generation_config.eos_token_id

model.generation_config

GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "max_length": 256,
  "pad_token_id": 50256
}

In [7]:

def tokenization_fn(captions, max_target_length):
    labels = tokenizer(captions, 
                      padding="max_length", 
                      max_length=max_target_length, truncation=True).input_ids

    return labels

def feature_extraction_fn(images):
    encoder_inputs = feature_extractor(images=images, return_tensors="pt")
    return encoder_inputs.pixel_values

def preprocess_fn(examples, max_target_length, check_image = True):
    """Run tokenization + image feature extraction"""
    image_paths = examples['raw_image']
    captions = examples['caption']
    
    model_inputs = {}
    model_inputs['labels'] = tokenization_fn(captions, max_target_length)
    model_inputs['input_ids'] = model_inputs['labels']
    model_inputs['pixel_values'] = feature_extraction_fn(image_paths)

    return model_inputs

In [8]:
# processed_dataset = ds.map(
#     function=preprocess_fn,
#     batched=True,
#     fn_kwargs={"max_target_length": 128},
#     remove_columns=ds['train'].column_names
# )

# processed_dataset['val']

processed_dataset = load_from_disk('processed_dataset')

In [9]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    generation_max_length=256,
    generation_config=model.generation_config,
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir="./image-captioning-output",
)


In [10]:
import evaluate
metric = evaluate.load("rouge")

In [11]:
import nltk

nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    preds = [pred[0].tolist() for pred in preds]

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    assert len(decoded_preds) == len(decoded_labels)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return result

In [12]:
from transformers import default_data_collator, DataCollatorForSeq2Seq
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['val'],
    data_collator=default_data_collator,
)


c:\Users\achia\anaconda3\envs\llm\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
wandb.init(project='LLM_Project_ablation')

trainer.train()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: varadnev (varnevnlp2023). Use `wandb login --relogin` to force relogin


  0%|          | 0/88716 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.
  1%|          | 500/88716 [00:32<1:33:05, 15.79it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
Your generation config was originally created from the model config, but the model c

{'loss': 0.334, 'grad_norm': 1.51202392578125, 'learning_rate': 4.9718201902700756e-05, 'epoch': 0.02}


  1%|          | 1000/88716 [01:05<1:35:22, 15.33it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generat

{'loss': 0.2897, 'grad_norm': 0.694603443145752, 'learning_rate': 4.943640380540151e-05, 'epoch': 0.03}


...

100%|█████████▉| 88500/88716 [1:36:46<00:14, 15.32it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


{'loss': 0.214, 'grad_norm': 0.34743809700012207, 'learning_rate': 1.2173677803327473e-07, 'epoch': 2.99}


100%|█████████▉| 88715/88716 [1:37:01<00:00, 16.18it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For c

{'eval_loss': 0.22994141280651093, 'eval_rouge1': 0.17395024220231126, 'eval_rouge2': 0.0, 'eval_rougeL': 0.17388894913056713, 'eval_rougeLsum': 0.17393498375680405, 'eval_runtime': 40.1185, 'eval_samples_per_second': 124.631, 'eval_steps_per_second': 31.158, 'epoch': 3.0}
{'train_runtime': 5861.2596, 'train_samples_per_second': 60.543, 'train_steps_per_second': 15.136, 'train_loss': 0.23003953524107562, 'epoch': 3.0}


TrainOutput(global_step=88716, training_loss=0.23003953524107562, metrics={'train_runtime': 5861.2596, 'train_samples_per_second': 60.543, 'train_steps_per_second': 15.136, 'train_loss': 0.23003953524107562, 'epoch': 3.0})

In [16]:
trainer.save_model("models/ablation")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
